# 📓 04 - Pipeline RAG usando LangChain `RetrievalQAChain`

Neste notebook, vamos:
✅ Usar FAISS como banco vetorial.  
✅ Integrar um LLM (OpenAI API) para respostas contextuais.  
✅ Usar `RetrievalQAChain` para pipeline de RAG “oficial” do LangChain.

---

## ⚙️ Setup inicial

```python
# !pip install langchain langchain-community langchain-openai sentence-transformers faiss-cpu
# !pip install python-dotenv
```

In [1]:
import os
from dotenv import load_dotenv
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import TextLoader
from langchain.schema import Document
from langchain.chains import RetrievalQA

### 1️⃣ Configuração de ambiente

In [2]:
load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")

### 2️⃣ Preparar documentos e chunking

In [3]:
# Exemplo de textos/documentos
texts = [
    "Retrieval-Augmented Generation (RAG) combina recuperação de informações e geração de linguagem natural.",
    "A IA está revolucionando setores como saúde e finanças.",
    "Python é uma linguagem de programação muito popular e versátil."
]

# Documentos com LangChain
docs = [Document(page_content=txt) for txt in texts]

# Splitter para criar chunks menores
splitter = RecursiveCharacterTextSplitter(chunk_size=50, chunk_overlap=10)
docs_split = splitter.split_documents(docs)

print(f"🔹 {len(docs_split)} chunks criados.")

🔹 7 chunks criados.


### 3️⃣ Indexar no FAISS via LangChain

In [4]:
embeddings = OpenAIEmbeddings(api_key=openai_api_key)
vectorstore = FAISS.from_documents(docs_split, embeddings)

print("🔹 FAISS indexado com LangChain!")

🔹 FAISS indexado com LangChain!


### 4️⃣ Criar RetrievalQAChain

In [5]:
# Inicializar LLM
llm = ChatOpenAI(api_key=openai_api_key, model="gpt-3.5-turbo")  #"gpt-4o")

# Criar Chain
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",  # "stuff" = coloca tudo no prompt
    retriever=vectorstore.as_retriever(search_kwargs={"k": 2}),
    return_source_documents=True
)

### 5️⃣ Testar o pipeline com qa_chain

In [6]:
query = "Como a IA está mudando a indústria?"

result = qa_chain({"query": query})

print("🔹 Resposta final:\n", result["result"])
print("\n🔹 Documentos recuperados:")
for doc in result["source_documents"]:
    print("-", doc.page_content)

C:\Users\vinicius\AppData\Local\Temp\ipykernel_16448\2154320861.py:3: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = qa_chain({"query": query})


🔹 Resposta final:
 A IA está revolucionando setores como saúde e linguagem natural.

🔹 Documentos recuperados:
- A IA está revolucionando setores como saúde e
- de linguagem natural.


### Extras

### Adicionando metadados aos documentos

In [7]:
from langchain.schema import Document

# Agora cada documento tem metadados
docs = [
    Document(
        page_content="Retrieval-Augmented Generation (RAG) combina recuperação de informações e geração de linguagem natural.",
        metadata={"source": "Blog IA", "date": "2024-01-10"}
    ),
    Document(
        page_content="A IA está revolucionando setores como saúde e finanças.",
        metadata={"source": "Revista Tech", "date": "2024-03-22"}
    ),
    Document(
        page_content="Python é uma linguagem de programação muito popular e versátil.",
        metadata={"source": "Blog Python", "date": "2023-11-05"}
    )
]

### Splitter + FAISS com metadados

In [8]:
splitter = RecursiveCharacterTextSplitter(chunk_size=50, chunk_overlap=10)
docs_split = splitter.split_documents(docs)

embeddings = OpenAIEmbeddings(api_key=openai_api_key)
vectorstore = FAISS.from_documents(docs_split, embeddings)

print("🔹 FAISS indexado com LangChain e metadados!")

🔹 FAISS indexado com LangChain e metadados!


### Chain usando RetrievalQAChain com metadados

In [9]:
from langchain.chains import RetrievalQA

llm = ChatOpenAI(api_key=openai_api_key, model="gpt-3.5-turbo")  #"gpt-4o")

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectorstore.as_retriever(search_kwargs={"k": 2}),
    return_source_documents=True
)

### Consulta e exibição dos metadados dos chunks recuperados

In [10]:
query = "Como a IA está mudando as indústrias?"

result = qa_chain({"query": query})

print("🔹 Resposta final:\n", result["result"])

print("\n🔹 Documentos recuperados e metadados:")
for doc in result["source_documents"]:
    print(f"- Fonte: {doc.metadata['source']} (Data: {doc.metadata['date']})")
    print("  Texto:", doc.page_content)


🔹 Resposta final:
 A IA está revolucionando setores como saúde e finanças.

🔹 Documentos recuperados e metadados:
- Fonte: Revista Tech (Data: 2024-03-22)
  Texto: A IA está revolucionando setores como saúde e
- Fonte: Revista Tech (Data: 2024-03-22)
  Texto: saúde e finanças.


### Testando outros chain_types

O LangChain tem vários tipos de Chains que influenciam como o contexto é formatado:

| **chain_type** | **Descrição resumida**                                                                                   |
|-----------------|----------------------------------------------------------------------------------------------------------|
| stuff           | Insere todos os documentos no prompt (padrão, rápido e simples)                                          |
| map_reduce      | Divide em partes menores e combina as respostas (bom para documentos muito grandes)                      |
| refine          | Processa o 1º doc, depois vai “refinando” com os demais (iterativo, mais preciso)                        |

In [11]:
qa_chain_mapreduce = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="map_reduce",
    retriever=vectorstore.as_retriever(search_kwargs={"k": 2}),
    return_source_documents=True
)

result_mapreduce = qa_chain_mapreduce({"query": query})
print("\n🔹 Resposta com map_reduce:\n", result_mapreduce["result"])


🔹 Resposta com map_reduce:
 A IA está revolucionando diversas indústrias ao redor do mundo, melhorando a eficiência, a precisão, a automação e a personalização em setores como saúde e financeiro. Na área da saúde, ela está possibilitando diagnósticos mais precisos, análise de grandes conjuntos de dados e personalização de tratamentos. Já no setor financeiro, a IA está sendo usada para detectar fraudes, fazer previsões de mercado mais precisas e melhorar a experiência do cliente. Em resumo, a IA está transformando a maneira como as indústrias operam, tornando os processos mais eficientes, inteligentes e adaptáveis às demandas do mercado.


In [12]:
qa_chain_refine = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="refine",
    retriever=vectorstore.as_retriever(search_kwargs={"k": 2}),
    return_source_documents=True
)

result_refine = qa_chain_refine({"query": query})
print("\n🔹 Resposta com refine:\n", result_refine["result"])


🔹 Resposta com refine:
 A inteligência artificial (IA) está revolucionando diversas indústrias de diferentes maneiras. Além dos exemplos anteriores, é importante destacar o impacto da IA nos setores de saúde e finanças.

Na área da saúde, a IA está sendo amplamente utilizada para otimizar diagnósticos médicos, personalizar tratamentos com base em dados específicos de cada paciente, gerenciar de forma eficiente registros eletrônicos de saúde, prever surtos de doenças e até mesmo desenvolver novos medicamentos e terapias.

No setor financeiro, a IA tem sido fundamental para prever tendências de mercado, detectar fraudes em transações financeiras, automatizar processos de análise de risco, criar assistentes virtuais para atendimento ao cliente e até mesmo desenvolver algoritmos de negociação de alta frequência.

Esses avanços tecnológicos estão transformando significativamente as indústrias de saúde e finanças, proporcionando benefícios como maior eficiência, precisão nos diagnósticos e 